# Analisando os dados disponibilizados

### 1) Importar bibliotecas

In [ ]:
import pandas as pd
from unidecode import unidecode

### 2) Ler dados brutos

In [ ]:
escolas = pd.read_csv("dados_brutos/escolas.csv")

material_didatico = pd.read_csv("dados_brutos/material_didatico.csv")

subprefeituras = pd.read_csv("dados_brutos/subprefeituras.csv")

### 3) Criar cópias dos DFs

In [ ]:
subprefeituras_copia = subprefeituras.copy()

In [ ]:
material_didatico_copia = material_didatico.copy()

In [ ]:
escolas_copia = escolas.copy()

### 4) Tratamento dos DFs

#### 4.1) Tratamento geral

1) aplicar lower para todas as colunas
2) nome das colunas em snake_case
3) strings não devem conter acentos
4) todas as strings devem estar em maiúsculo

In [ ]:
def renomear_colunas(df):
    new_columns = {col: unidecode(col.lower().strip()) for col in df.columns}
    df.rename(columns=new_columns, inplace=True)
    for col in df.columns:
        df[col] = df[col].apply(lambda x: unidecode(str(x).upper())) if df[col].dtypes == 'object' else df[col]

In [ ]:
escolas_copia['endereco'] = escolas_copia['endereco'].apply(add_comma_to_address)

In [ ]:
escolas_copia['endereco'] = escolas_copia['endereco'].apply(replace_s_n)

In [ ]:
#  str.replace() para substituir vírgulas repetidas
condicao = escolas_copia['endereco'].str.count(',') > 1
escolas_copia['endereco'] = escolas_copia['endereco'].loc[condicao].str.replace(', ', ',')

In [ ]:
dfs = [escolas_copia, material_didatico_copia, subprefeituras_copia]

for df in dfs:
    renomear_colunas(df)
    display(df.head(5))

### 4.2) Tratamento para o DF material didático

#### O que deve ser feito:

1. Converter tipo coluna de objeto para inteiro

2. Substituir NaN e None por 0

3. Substituir '' e ' ' por 0

4. Substituir 'NaN' por 0

5. Converter valor numérico do tipo objeto para int (Ex.: '356' para 356)

Remover valores nulos


In [ ]:
material_didatico_copia['quantidade'].fillna('0', inplace=True)

Verifica valores diferentes de números

In [ ]:
for i, row in material_didatico_copia.iterrows():
    qnt = row['quantidade']
    if not qnt.isdigit():
        print(qnt)

Converte todos os valores da coluna para int (Modo 1)

In [ ]:
material_didatico_copia['quantidade'] = material_didatico_copia['quantidade'].apply(lambda x: int(x) if x.isdigit() else 0)

In [ ]:
# # Modo 2
# import re

# def normaliza_quantidade(valor) -> int:
#     if valor:
#         normaliza_valor = re.sub(r"[^\d: ]", "", valor.strip())
#         if normaliza_valor:
#             return int(normaliza_valor)
#         else:
#             return 0
#     else:
#         return 0
    
# material_didatico['quantidade'] = material_didatico['quantidade'].apply(lambda x: normaliza_quantidade(x))

Filtra dataframe com quantidade de material diferente de zero

In [ ]:
material_didatico_copia = material_didatico_copia[material_didatico_copia['quantidade'] != 0]

Verifica existência de valores nulos

In [ ]:
material_didatico_copia.isna().sum()

### 4.3) Tratamento para o DF subprefeituras

Verifica existência de valores duplicados

In [ ]:
subprefeituras_copia.duplicated().sum()

### 4.4) Tratamento para o DF escola

Normaliza coluna lat com 5 casas decimais

In [ ]:
for col in ['lat', 'lon']:
    escolas_copia[col] = escolas_copia[col].str.replace(",", ".").astype(float).round(5)

##### Substitui abreviações de endereço

In [ ]:
# verifica possíveis abreviações
abreviacoes = set()

for i, linha_escola in escolas_copia.iterrows():
    lista_palavra_endereco = linha_escola['endereco'].split()
    for palavra in lista_palavra_endereco:
        if len(palavra) <= 3 and not palavra.isdigit():
            abreviacoes.update([palavra])

abreviacoes

In [ ]:
# verifica possíveis abreviações para logradouros
filtro_abreviacoes = [abreviacao for abreviacao in abreviacoes if abreviacao.startswith('R') or abreviacao.startswith('AV')]
filtro_abreviacoes

In [ ]:
escolas_copia['endereco'] = escolas_copia['endereco'].str.replace('R. ', 'RUA ', regex=True).replace('AV. ', 'AVENIDA ', regex=True).replace('AV ', 'AVENIDA ', regex=True)

In [ ]:
escolas_copia

Valida normalização de abreviações do campo endereco

In [ ]:
for i, linha_escola_2 in escolas_copia.iterrows():
    if linha_escola_2['endereco'].startswith('R. ')  or \
       linha_escola_2['endereco'].startswith('AV. ') or \
       linha_escola_2['endereco'].startswith('AV '):
        print(linha_escola_2)

Adiciona vírgula para o número do endereço e unifica os SN

In [ ]:
# verifica possíveis abreviações
numeros = list()
enderecos_sem_numeros = list()

numeros = [linha_escola['endereco'].split()[-1] for i, linha_escola in escolas_copia.iterrows()]

for item in numeros:
    if not item.isdigit():
        enderecos_sem_numeros.append(item)

set_enderecos_sem_numeros = set(enderecos_sem_numeros)
print(set_enderecos_sem_numeros)

In [201]:
# utiliza apenas os sem números
for item in set_enderecos_sem_numeros:
    if item.startswith('S'):
        print(item)

S/N.deg
S/No
S/Ndeg
S/N
S/N.o


In [202]:
import re
# Função para adicionar vírgula entre endereço e número
def add_comma_to_address(address):
    # Use expressões regulares para encontrar o número no final do endereço
    match = re.search(r'(\d+(\.\d+)?)$', address)
    
    if match:
        # Se um número for encontrado, adicione uma vírgula antes dele
        address_with_comma = re.sub(r'(\d+(\.\d+)?)$', r', \1', address)
        return address_with_comma
    else:
        return address

# Função para tratar as diversas maneiras de endereços sem numero para `SN``
def replace_s_n(text):
    # Lista de trechos a serem substituídos
    to_replace = ["S/N.deg", "S/No", "S/Ndeg", "S/N", "S/N.o"]
    # to_replace = ["S/No", "S/N.o", "S/N", "S/N.deg", "S/Ndeg", "SNdeg", "SN.deg"]
    
    # Substituir os trechos na string
    for item in to_replace:
        text = text.replace(item, ",SN")
    
    return text

In [203]:
def renomear_colunas(df):
    new_columns = {col: unidecode(col.lower().strip()) for col in df.columns}
    df.rename(columns=new_columns, inplace=True)
    for col in df.columns:
        df[col] = df[col].apply(lambda x: unidecode(str(x).upper())) if df[col].dtypes == 'object' else df[col]